In [1]:
%matplotlib widget
import cdflib
import calendar
import time
import numpy as np
import matplotlib.pyplot as plt
import datetime

file = "../20170810data/mms1/fgm/mms1_fgm_brst_l2_20170810121733_v5.99.0.cdf"
var = "mms1_fgm_b_gsm_brst_l2"
_EPOCH_ORD = datetime.date(1970, 1, 1).toordinal()

In [2]:
def timegm(tuple):
    """Calculate Unix timestamp from GMT."""
    year, month, day, hour, minute, second, nano = tuple[:7]
    days = datetime.date(year, month, 1).toordinal() - _EPOCH_ORD + day - 1
    hours = days*24 + hour
    minutes = hours*60 + minute
    seconds = minutes*60 + second
    nanos = seconds*1e9 + nano
    return nanos

def get_cdf_var(filename, varnames):
  '''
  Privately used function.
  pulls particular variables from a CDF
  note: if variable has more than one set of data (E.g. b-field with x,y,z
  components) it will be necessary to format the data by reshaping the array
  from a 1D to a 2D array
  (may find workaround/better way later)
  Inputs:
      filename- string of the complete path to the specified CDF file
      varnames- list of strings which contain the CDF variables that are
          to be extracted
  Outputs:
      data- list of numpy arrays containing the desired variables' data
  '''
  cdf_file=cdflib.CDF(filename,varnames)
  data=[]
  for varname in varnames:
      var_data=np.array(cdf_file.varget(varname))
      data.append(var_data)
  return data

def getData(file, timeInterval, axis):
    '''
    Gets relevant processed data from CDF file.
    Inputs:
        file- filename taken from array "files"
        timeInterval- period for data
        axis- String value, "x","y","z", or "all" (a vector summation of all components)
    Outputs:
        x- horizontal axis data, in this case Epoch (ns)
        y- veritcal axis data, in this case B-field measurement (nT)
    '''
    CDFfile = cdflib.CDF(file[0])
    filename = file[0]
    varname = file[1]

    #generate time series
    raw_times = get_cdf_var(filename, ["Epoch"])[0]
    times = []
    
    #time of event
    year = 2017
    month = 8
    day = 10
    hour = 12

    start_minute = timeInterval[0]
    start_sec = timeInterval[1]
    start_nano = timeInterval[2]
    stop_minute = timeInterval[3]
    stop_sec = timeInterval[4]
    stop_nano = timeInterval[5]
    
    #algorithm to find indexes of given period start and end, quickly
    #Note that 'epoch' is 1/1/2000 12:00:00 (MMS epoch) J2000, also known as TT2000
    start_epoch_nano = timegm((year-30, month, day+1, hour-12, start_minute+1, start_sec+9, start_nano))
    stop_epoch_nano = timegm((year-30, month, day+1, hour-12, stop_minute+1, stop_sec+9, stop_nano))
    start_index_epoch_nano = raw_times[0]
    stop_index_epoch_nano = raw_times[-1]
    time_between_measurements = (stop_index_epoch_nano - start_index_epoch_nano) / (len(raw_times) - 1)
    est_start_index = round((start_epoch_nano - start_index_epoch_nano) / time_between_measurements)
    est_stop_index = round((stop_epoch_nano - start_index_epoch_nano) / time_between_measurements)
    est_start_indexes = np.arange(est_start_index - 5, est_start_index + 5, 1).tolist()
    est_stop_indexes = np.arange(est_stop_index - 5, est_stop_index + 5, 1).tolist()
    
    for i in est_start_indexes:
        i_epoch_nano = round(raw_times[i] / 1e9)
        if i_epoch_nano + 50000000 > start_epoch_nano or i_epoch_nano - 50000000 < start_epoch_nano:
            start_index = i
    for j in est_stop_indexes:
        j_epoch_nano = round(raw_times[j] / 1e9)
        if j_epoch_nano + 50000000 > stop_epoch_nano or j_epoch_nano - 50000000 < stop_epoch_nano:
            stop_index = j

    raw_data = get_cdf_var(filename, [varname])[0]

    if axis == "all": #vector addition
        Bx = []
        By = []
        Bz = []
        
        for i in range(0,len(raw_data)):
            Bx.append(raw_data[i][0]) # x data
            By.append(raw_data[i][1]) # y data
            Bz.append(raw_data[i][2]) # z data

        Bmag = vectorAdd(Bx, By, Bz)
        
        #data sets as arrays, len = 2048
        x = raw_times[start_index:stop_index].tolist()
        y = Bmag[start_index:stop_index]
        
    else:
        data = []
            
        coordinateSystem = {"x":0, "y":1, "z":2}
        coordinate = coordinateSystem.get(axis)

        for i in range(0,len(raw_data)):
            data.append(raw_data[i][coordinate])

        #data sets as arrays, len = 2048
        x = raw_times[start_index:stop_index].tolist()
        y = data[start_index:stop_index]
        
    return x, y

In [9]:
#get data
time = [18, 50, 0, 18, 55, 0]
fgmBx = getData([file, var], time, "x")[1]
fgmBxavg = sum(fgmBx) / len(fgmBx)
for i in range(0, len(fgmBx)):
    fgmBx[i] -= fgmBxavg
fgmBy = getData([file, var], time, "y")[1]
fgmByavg = sum(fgmBy) / len(fgmBy)
for i in range(0, len(fgmBy)):
    fgmBy[i] -= fgmByavg
fgmBz = getData([file, var], time, "z")[1]
fgmBzavg = sum(fgmBz) / len(fgmBz)
for i in range(0, len(fgmBz)):
    fgmBz[i] -= fgmBzavg
    
#plotting
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
"""
xy = ax.plot(fgmBx, fgmBy, label = "xy")
xz = ax.plot(fgmBx, fgmBz, label = "xz")
yz = ax.plot(fgmBy, fgmBz, label = "yz")"""
ax.plot3D(fgmBx, fgmBy, fgmBz, label = "xyz")
ax.set_xlabel("Bx")
ax.set_ylabel("By")
ax.set_zlabel("Bz")
plt.title(str(time[0]) + ":" + str(time[1]) + "." + str(time[2]) + " to " + str(time[3]) + ":" + str(time[4]) + "." + str(time[5]))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …